### Jul AdventKalender D17

https://adventofcode.com/2022/day/17

In [1]:
import numpy as np

#### Day 17.1 

Make a Tetris!

Given five types of blocks that fall in order (from left to right, 1234512345...):

      1          2           3             4            5
    ####        .#.         ..#            #            ##          
                ###         ..#            #            ##
                .#.         ###            #            
                                           #

The blocks don't spin like in Tetris, but they get pushed left(**<**) or right(**>**) according to the input file which looks like this:

    >>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>

The play board is **7 units wide**. Each block appears with **2 units** away from the left wall and **3 units** above the highest block/floor.

After a block appears, it alternates between being **pushed** 1 unit to the left/right and then **falling** 1 unit down. The push doesn't take effect if the block will move into the walls, floor, or a stopped block. The block **stops** when it falls on the floor or an already-fallen block, and a **new** block immediately appears.

How tall will the tower of blocks be after **2022** blocks have stopped falling?

In [2]:
def readFile(file_name):
    data_force = {}
    f = open(file_name, "r")
    data_force = f.read().strip()
    f.close()
    return data_force

In [3]:
def _blockGeneratePos(block_arr, height_top):
    global board
    gen_height = (4 + height_top) * 1j
    gen_pos = block_arr + gen_height
    return gen_pos

def _blockPushStep(force, block_pos, height_top):
    global board
    block_pushed = block_pos+force
    if min(block_pushed.real) < 0 or max(block_pushed.real) > 6: # out of border, pushed into walls
        return block_pos
    if height_top < min(block_pushed.imag):
        return block_pushed
    for i in range(len(block_pushed)):
        if board[int(block_pushed[i].imag),int(block_pushed[i].real)]==1:# pushed into block
            return block_pos
    return block_pushed

def _blockFallStep(block_pos, height_top):
    global board
    block_fell = block_pos-1j
    if min(block_fell.imag) == 0: # falls into floor
        return True, block_pos
    if height_top < min(block_fell.imag): # no need to check, can fall
        return False, block_fell
    for i in range(len(block_fell)):
        if board[int(block_fell[i].imag),int(block_fell[i].real)]==1: # fell into block
            return True, block_pos
    return False, block_fell

In [4]:
def blockOps(total_blocks):
    global data_force, blocks, board
    id_b = 0
    id_f = 0
    height_top = 0
    len_f = len(data_force)
    board = np.zeros((total_blocks*2, 7))
    pattern = {}
    for i in range(total_blocks):
        block_cur = blocks[id_b%5]
        block_cur_pos = _blockGeneratePos(block_cur, height_top)
        while True:
            id_f %= len_f
            force_cur = data_force[id_f]
            pattern[id_f] = [i] if id_f not in pattern else pattern[id_f]+[i]
            block_cur_pos = _blockPushStep(force_cur, block_cur_pos, height_top)
            #print('after pushed by',force_cur,':',block_cur_pos)
            isStop, block_cur_pos = _blockFallStep(block_cur_pos, height_top)
            #print('after falling:',block_cur_pos, 'is stopped:',isStop)
            id_f += 1
            if isStop: 
                board[block_cur_pos.imag.astype(int),block_cur_pos.real.astype(int)] = 1
                height_top = max(height_top, max(block_cur_pos.imag))
                #print(block_cur_pos, height_top)
                break
        id_b += 1
    return height_top, pattern

In [5]:
total_blocks = 2022

# read force
data_force = readFile('data/input17.txt')
data_force = [1 if push == '>' else -1 for push in data_force]

# use complex number to represent space
# real: row index (-2 to 4 : 7 units); imaginary: col index bottom up (floor=0)
# define blocks 
blocks = [np.array([0,1,2,3])+2, np.array([1j,1,1+1j,1+2j,2+1j])+2, 
          np.array([0,1,2,2+1j,2+2j])+2, np.array([0,1j,2j,3j])+2, np.array([0,1,1j,1+1j])+2]
# define game board
#         gen
#    0  1  2  3  4  5  6
# 3  .  .  .  .  .  .  .
# 2  .  .  .  .  .  .  .
# 1  .  .  .  .  .  .  .
# 0---------------------  floor
# (nrow, 7) # bottom up, floor=0(ignore)

# operate
height_top,_ = blockOps(total_blocks)
height_top

3055.0

#### Day 17.2

How tall will the tower be after 1000000000000 rocks have stopped?

In [6]:
# Find a repeated pattern, so that we don't need to iterate 1000000000000 times
# thinking start with the force id

max_iteration = len(data_force)*5 #n_force x n_block
board = np.zeros((max_iteration*2, 7))
_, pattern = blockOps(max_iteration)

In [7]:
id_f_max = max([i for i,_ in pattern.items()]) # the largest id_f
print('The largest excuted force id is', id_f_max, 'and it is excuted', len(pattern[id_f_max]), 'times.')
rep_iters = np.unique(np.array(pattern[id_f_max][1:]) - np.array(pattern[id_f_max][:-1]))
print('It follows {0} pattern to repeat: every {1} blocks'.format(len(rep_iters), rep_iters))

The largest excuted force id is 10090 and it is excuted 29 times.
It follows 1 pattern to repeat: every [1690] blocks


In [8]:
height_1,_ = blockOps(pattern[id_f_max][1])
height_2,_ = blockOps(pattern[id_f_max][2])
height_3,_ = blockOps(pattern[id_f_max][3])
print('The heights by operating the first three numbers of blocks are', height_1, height_2, height_3)
print('And their distance is', height_2-height_1, height_3-height_2)

The heights by operating the first three numbers of blocks are 5085.0 7633.0 10181.0
And their distance is 2548.0 2548.0


In [9]:
rep_iters = rep_iters[0]
rep_height = height_2-height_1
total_blocks = 1000000000000
base_iters = total_blocks%rep_iters + max_iteration//rep_iters*rep_iters

In [10]:
height_top,_ = blockOps(base_iters)
assert((total_blocks-base_iters)%rep_iters==0), "Fail, should be divisible by repeated iterations"
height_top += rep_height * (total_blocks-base_iters)//rep_iters
height_top

1507692307690.0